In [4]:
import pandas as pd

# читаем данные из таблиц
events_df = pd.read_csv('data/7_4_Events.csv')
purchase_df = pd.read_csv('data/purchase.csv')

# находим список пользователей, зарегистрировавшихся в 2018
cond = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
registered = events_df[cond]['user_id'].to_list() 

# извлекаем пользователей, зарегистрировавшихся в 2018 году
events_df = events_df[events_df.user_id.isin(registered)] 
events_df.start_time = pd.to_datetime(events_df.start_time, format='%Y-%m-%dT%H:%M:%S')
purchase_df = purchase_df[purchase_df.user_id.isin(registered)]
purchase_df.event_datetime = pd.to_datetime(purchase_df.event_datetime, format='%Y-%m-%dT%H:%M:%S')

# переиеновываем столбцы id
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})
# объединяем датафреймы, сбрасываем индексы и сортируем по возрастанию времени
total_events_df = pd.concat([events_df,purchase_df],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

# делим пользователей по выбранному уровню сложности 
users_select_level_easy = total_events_df[total_events_df["selected_level"] == "easy"]["user_id"].unique()
users_select_level_medium = total_events_df[total_events_df["selected_level"] == "medium"]["user_id"].unique()
users_select_level_hard = total_events_df[total_events_df["selected_level"] == "hard"]["user_id"].unique()

# выделим датафреймы по оплатам пользователей, в зависимости от уровня сложности
purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_select_level_easy)]
purchase_df_2 = purchase_df[purchase_df["user_id"].isin(users_select_level_medium)]
purchase_df_3 = purchase_df[purchase_df["user_id"].isin(users_select_level_hard)]

# находим процент оплативших пользователей, в зависимости от уровня сложности
percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / len(users_select_level_easy)
print(
    "Процент пользователей, которые выбрали уровень easy и оплатили вопросы: {:.2%}".format(
        percent_of_purchase_1
    )
)
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(users_select_level_medium)
print(
    "Процент пользователей, которые выбрали уровень medium и оплатили вопросы: {:.2%}".format(
        percent_of_purchase_2
    )
)
percent_of_purchase_3 = purchase_df_3["user_id"].nunique() / len(users_select_level_hard)
print(
    "Процент пользователей, которые выбрали уровень hard и оплатили вопросы: {:.2%}".format(
        percent_of_purchase_3
    )
)
# выделим датафреймы с разными событиями выбора уровня сложности
select_easy_df = total_events_df[total_events_df['selected_level'] == 'easy']
select_easy_df = select_easy_df[["user_id", "start_time"]].rename(columns={"start_time": "select_time"})
select_medium_df = total_events_df[total_events_df['selected_level'] == 'medium']
select_medium_df = select_medium_df[["user_id", "start_time"]].rename(columns={"start_time": "select_time"})
select_hard_df = total_events_df[total_events_df['selected_level'] == 'hard']
select_hard_df = select_hard_df[["user_id", "start_time"]].rename(columns={"start_time": "select_time"})

# объединяем датафреймы в рамках одной группы и находим разницу во времени между выбором уровня и оплатой
merged_df_1 = select_easy_df.merge(purchase_df_1, on="user_id", how="inner")
merged_df_1["timedelta"] = (merged_df_1["event_datetime"] - merged_df_1["select_time"])
merged_df_2 = select_medium_df.merge(purchase_df_2, on="user_id", how="inner")
merged_df_2["timedelta"] = (merged_df_2["event_datetime"] - merged_df_2["select_time"])
merged_df_3 = select_hard_df.merge(purchase_df_3, on="user_id", how="inner")
merged_df_3["timedelta"] = (merged_df_3["event_datetime"] - merged_df_3["select_time"])
print("Среднее время между выбором уровня сложности и оплатой (для easy): ", merged_df_1["timedelta"].mean())
print("Среднее время между выбором уровня сложности и оплатой (для medium): ", merged_df_2["timedelta"].mean())
print("Среднее время между выбором уровня сложности и оплатой (для hard): ", merged_df_3["timedelta"].mean())

# выделим датафрейм с событием registration
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df = registration_df[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})

# объединяем датафреймы в рамках одной группы и находим разницу во времени между регистрацией и оплатой
merged_df_4 = purchase_df_1.merge(registration_df, on="user_id", how="inner")
merged_df_4["timedelta"] = (merged_df_4["event_datetime"] - merged_df_4["registration_time"])
merged_df_5 = purchase_df_2.merge(registration_df, on="user_id", how="inner")
merged_df_5["timedelta"] = (merged_df_5["event_datetime"] - merged_df_5["registration_time"])
merged_df_6 = purchase_df_3.merge(registration_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["event_datetime"] - merged_df_6["registration_time"])
print("Среднее время между регистрацией и оплатой (для easy): ", merged_df_4["timedelta"].mean())
print("Среднее время между регистрацией и оплатой (для medium): ", merged_df_5["timedelta"].mean())
print("Среднее время между регистрацией и оплатой (для hard): ", merged_df_6["timedelta"].mean())

Процент пользователей, которые выбрали уровень easy и оплатили вопросы: 7.72%
Процент пользователей, которые выбрали уровень medium и оплатили вопросы: 20.86%
Процент пользователей, которые выбрали уровень hard и оплатили вопросы: 35.39%
Среднее время между выбором уровня сложности и оплатой (для easy):  3 days 14:58:52.941798941
Среднее время между выбором уровня сложности и оплатой (для medium):  3 days 23:14:13.165118679
Среднее время между выбором уровня сложности и оплатой (для hard):  3 days 07:20:41.420814479
Среднее время между регистрацией и оплатой (для easy):  3 days 22:10:23.211640211
Среднее время между регистрацией и оплатой (для medium):  4 days 06:12:06.576883384
Среднее время между регистрацией и оплатой (для hard):  3 days 14:55:19.257918552


Наибольший процент пользователей, которые оплатили вопросы, выбирают уровень сложности hard. Следовательно вероятность оплаты зависит от выбранного уровня сложности. 

Наименьшее среднее время между событиями выбор уровня сложности - покупка вопросов и регистрация - покупка вопросов, зафиксировано для пользователей, выбравших уровень сложности hard. Это показывает их болеее высокую заинтересованность в дальнейшем использовании приложения. 